In [33]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [34]:
!pip install matplotlib numpy Pillow tensorflow opencv-python mediapipe

In [150]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import cv2
import mediapipe as mp
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.python.keras.layers import Dense, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.preprocessing import image as keras_image

###**Model**

In [151]:
def identity_block(x, filter):
    # copy tensor to variable called x_skip
    x_skip = x
    # Layer 1
    x = tf.keras.layers.Conv2D(filter, (3,3), padding = 'same')(x)
    x = tf.keras.layers.BatchNormalization(axis=3)(x)
    x = tf.keras.layers.Activation('relu')(x)
    # Layer 2
    x = tf.keras.layers.Conv2D(filter, (3,3), padding = 'same')(x)
    x = tf.keras.layers.BatchNormalization(axis=3)(x)
    # Add Residue
    x = tf.keras.layers.Add()([x, x_skip])
    x = tf.keras.layers.Activation('relu')(x)
    return x

In [152]:
def convolutional_block(x, filter):
    # copy tensor to variable called x_skip
    x_skip = x
    # Layer 1
    x = tf.keras.layers.Conv2D(filter, (3,3), padding = 'same', strides = (2,2))(x)
    x = tf.keras.layers.BatchNormalization(axis=3)(x)
    x = tf.keras.layers.Activation('relu')(x)
    # Layer 2
    x = tf.keras.layers.Conv2D(filter, (3,3), padding = 'same')(x)
    x = tf.keras.layers.BatchNormalization(axis=3)(x)
    # Processing Residue with conv(1,1)
    x_skip = tf.keras.layers.Conv2D(filter, (1,1), strides = (2,2))(x_skip)
    # Add Residue
    x = tf.keras.layers.Add()([x, x_skip])
    x = tf.keras.layers.Activation('relu')(x)
    return x

In [153]:
def ResNet34(shape = (180, 180, 3), classes = 2):
    # Step 1 (Setup Input Layer)
    x_input = tf.keras.layers.Input(shape)
    x = tf.keras.layers.ZeroPadding2D((3, 3))(x_input)
    # Step 2 (Initial Conv layer along with maxPool)
    x = tf.keras.layers.Conv2D(64, kernel_size=7, strides=2, padding='same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)
    x = tf.keras.layers.MaxPool2D(pool_size=3, strides=2, padding='same')(x)
    # Define size of sub-blocks and initial filter size
    block_layers = [3, 4, 6, 3]
    filter_size = 64
    # Step 3 Add the Resnet Blocks
    for i in range(4):
        if i == 0:
            # For sub-block 1 Residual/Convolutional block not needed
            for j in range(block_layers[i]):
                x = identity_block(x, filter_size)
        else:
            # One Residual/Convolutional Block followed by Identity blocks
            # The filter size will go on increasing by a factor of 2
            filter_size = filter_size*2
            x = convolutional_block(x, filter_size)
            for j in range(block_layers[i] - 1):
                x = identity_block(x, filter_size)
    # Step 4 End Dense Network
    x = tf.keras.layers.AveragePooling2D((2,2), padding = 'same')(x)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(512, activation = 'relu')(x)
    x = tf.keras.layers.Dense(1, activation = 'sigmoid')(x)
    model = tf.keras.models.Model(inputs = x_input, outputs = x, name = "ResNet34")
    return model

In [154]:
model = ResNet34()

In [155]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [156]:
model.load_weights('/content/drive/MyDrive/rope_skipping/runs/2000-epochs/weights/best.h5')

###**MediaPipe**

In [194]:
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)

In [195]:
cap = cv2.VideoCapture('/content/y2mate.com - How To Jump Rope  Beginner Tips_1080p.mp4')

In [196]:
output_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
output_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter('output_video.mp4', cv2.VideoWriter_fourcc(*'mp4v'), 30, (output_width, output_height))

In [197]:
def preprocess_frame(frame):
    # Resize the frame to match the model's input size
    img = cv2.resize(frame, (180, 180))
    # Convert BGR image to RGB
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    # Normalize pixel values to the range [0, 1]
    img_normalized = img_rgb / 255.0
    # Add batch dimension to match model input shape
    img_batch = np.expand_dims(img_normalized, axis=0)
    return img_batch

In [198]:
def extract_keypoints(frame):
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(image)
    if results.pose_landmarks:
        return [(lm.x, lm.y, lm.z) for lm in results.pose_landmarks.landmark]
    else:
        return None

In [199]:
def prediction(frame):
    # Convert the frame to an image array
    img_array = keras_image.array_to_img(frame)  # Use the 'image' submodule from Keras
    img_array = img_array.resize((180, 180))  # Resize to match the target size
    img_array = keras_image.img_to_array(img_array)
    img_array = np.expand_dims(img_array, axis=0)

    # Perform prediction
    predictions = model.predict(img_array)
    predicted_class = 1 if predictions[0][0] >= 0.5 else 0
    class_label = 'Skipping' if predicted_class == 1 else 'Not Skipping'
    return class_label

In [200]:
def write_text(frame, text):
    font = cv2.FONT_HERSHEY_SIMPLEX
    org = (50, 50)
    fontScale = 1
    color = (0, 0, 255)
    thickness = 2
    cv2.putText(frame, text, org, font, fontScale, color, thickness, cv2.LINE_AA)

In [201]:
specific_keypoints_indices = [23, 24, 25, 26, 27, 28, 31, 32, 13, 14]

In [202]:
all_keypoints_detected_flag = False

# Iterate over frames
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Resize frame and process for keypoints
    frame = cv2.resize(frame, (output_width, output_height))
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(image)
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    keypoints = extract_keypoints(frame)

    if keypoints:
        keypoints_detected = all(keypoints[idx] is not None for idx in specific_keypoints_indices)

        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
                                  mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                )

        if keypoints_detected:
            all_keypoints_detected_flag = True
            print("Key Points Detected")

            # If all keypoints are detected, pass the frame to the model for prediction
            result = prediction(frame)  # Pass the original frame for prediction, not the image
            print("Predicted class:", result)

            write_text(image, result)

        out.write(image)

    else:
        # Draw landmarks on the frame without keypoints
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
                                  mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                )

        write_text(image, 'Keypoints Not Detected')

        out.write(image)

# Release resources
cap.release()
out.release()
print("Done processing video")

# Release MediaPipe Pose model
pose.close()

Key Points Detected
1/1 [==============================] - 0s 155ms/step
Predicted class: Skipping
Key Points Detected
1/1 [==============================] - 0s 148ms/step
Predicted class: Skipping
Key Points Detected
1/1 [==============================] - 0s 142ms/step
Predicted class: Skipping
Key Points Detected
1/1 [==============================] - 0s 138ms/step
Predicted class: Skipping
Key Points Detected
1/1 [==============================] - 0s 138ms/step
Predicted class: Skipping
Key Points Detected
1/1 [==============================] - 0s 154ms/step
Predicted class: Skipping
Key Points Detected
1/1 [==============================] - 0s 157ms/step
Predicted class: Skipping
Key Points Detected
1/1 [==============================] - 0s 151ms/step
Predicted class: Skipping
Key Points Detected
1/1 [==============================] - 0s 143ms/step
Predicted class: Skipping
Key Points Detected
1/1 [==============================] - 0s 145ms/step
Predicted class: Skipping
Key Points